# NN by Hand

In [1]:
# Using NumPy 1.18 and Sklearn 0.24

import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

We're going to make a very simple neural network by hand.

Let's imagine that we have just a five-neuron network: two input neurons, two hidden-layer neurons, and a single output neuron.

We'll start by giving ourselves a regression problem.

In [2]:
X, y = make_regression(n_features=2,
                       n_informative=1,
                       noise=100,
                       random_state=42)

Let's first check to see how a vanilla linear regression model does:

In [3]:
LinearRegression().fit(X, y).score(X, y)

0.4144461358278726

## Input Neurons

We have just two features, so we'll have two input neurons.

In [4]:
n_samples = X.shape[0]
n_features = X.shape[1]

# Initial inputs
input1 = X[:, 0].reshape(-1, 1)
input2 = X[:, 1].reshape(-1, 1)

## Weights for the Connections Between the Input Neurons and the Hidden Neurons

We'll just start by setting our weights randomly. The idea will be that we'll be able to use gradient descent to improve their values during network training.

In [5]:
# Four weights to optimize for between input
# and hidden layers

# For simplicity let's assume biases of 0
# throughout

np.random.seed(42)

in_hid_weights1 = np.random.rand(2)
in_hid_weights2 = np.random.rand(2)

In [6]:
in_hid_weights1

array([0.37454012, 0.95071431])

In [7]:
in_hid_weights2

array([0.73199394, 0.59865848])

## Hidden Neurons

Each neuron in the hidden layer will take the two inputs and multiply them by a unique set of weights:

In [8]:
in1_to_hid = (np.sum(np.product([in_hid_weights1, X], axis=0),
                     axis=1)).reshape(-1, 1)
in2_to_hid = (np.sum(np.product([in_hid_weights2, X], axis=0),
                     axis=1)).reshape(-1, 1)

For simplicity let's assume a **linear activation function**, $f(x)=x$, in the hidden nodes.

## Weights for the Connections Between the Hidden Neurons and the Output Neuron

Again we'll just set our weights randomly. Here there will be two weights: one governing the connection between each hidden neuron and the output neuron.

In [9]:
np.random.seed(43)

hid_out_weights = np.random.rand(2)

In [10]:
hid_out_weights

array([0.11505457, 0.60906654])

## Output Neuron

Now we need to take the contribution from each hidden neuron and create a linear sum with the predetermined weights, just as above in the hidden neurons.

In [11]:
joint_to_out = np.hstack((in1_to_hid, in2_to_hid))

In [12]:
output = (np.sum(np.product([hid_out_weights, joint_to_out], axis=0),
                 axis=1)).reshape(-1, 1)

Again we'll assume a linear activation.

## Error Calculation

Now we can compute a measure of error:

In [13]:
output = output.flatten()

np.sqrt(mse(y, output))

132.15409393186755

### Backpropagation

Now then: How do we make corrections to our weights to improve our model's performance? Our network looks like this:

![nn](images/SimpleNN.png)

Clearly our output is a function of these six weights. But what function, exactly?

- In the top hidden node we construct: <br/> $H_1 = w_1X_1 + w_3X_2$ <br/> and in the bottom node we construct: <br/> $H_2 = w_2X_1 + w_4X_2$.
<br/> <br/>
- In the output node we construct: <br/> $\hat{y} = w_5H_1 + w_6H_2$ <br/> i.e. <br/> $\hat{y} = X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)$.

#### Loss Function

Our loss function (let's assume) is just $\mathcal{L} = \Sigma\left(y - \hat{y}\right)^2$. What are the partial derivatives of this function?

We have $\mathcal{L} = \Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)^2$.

Therefore:

Our loss function (let's assume) is just $\mathcal{L} = \Sigma\left(y - \hat{y}\right)^2$. What are the partial derivatives of this function?

We have $\mathcal{L} = \Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)^2$.

Therefore:

- $\tiny\frac{\partial\mathcal{L}}{\partial w_1} = -2w_5X_1\Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)$

- $\tiny\frac{\partial\mathcal{L}}{\partial w_2} = -2w_6X_1\Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)$

- $\tiny\frac{\partial\mathcal{L}}{\partial w_3} = -2w_5X_2\Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)$

- $\tiny\frac{\partial\mathcal{L}}{\partial w_4} = -2w_6X_2\Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)$

- $\tiny\frac{\partial\mathcal{L}}{\partial w_5} = -2(w_1X_1 + w_3X_2)\Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)$

- $\tiny\frac{\partial\mathcal{L}}{\partial w_6} = -2(w_2X_1 + w_4X_2)\Sigma\left(y - [X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)]\right)$

So the goal now should just be to nudge each of our weights in (the opposites of) these directions.

Let's first isolate each of these six weights:

In [14]:
w1 = in_hid_weights1[0]
w2 = in_hid_weights1[1]
w3 = in_hid_weights2[0]
w4 = in_hid_weights2[1]
w5 = hid_out_weights[0]
w6 = hid_out_weights[1]

Let's build an SGD function that will adjust weights after each training sample runs through the network.

In [15]:
def stoch_grad_desc(pred1=input1, pred2=input2, y=y, w1=w1, w2=w2, w3=w3,
                    w4=w4, w5=w5, w6=w6, times_thru=1, lr=1e-4):
    for k in range(times_thru):
        for j in range(pred1.shape[0]):
            in1_val = pred1[j]
            in2_val = pred2[j]
            sum_ = y[j] - in1_val*(w1*w5 + w2*w6)\
                - in2_val*(w3*w5 + w4*w6)
            w5 += lr*(w1*in1_val + w3*in2_val)*sum_
            w6 += lr*(w2*in1_val + w4*in2_val)*sum_
            w1 += lr*w5*in1_val*sum_
            w2 += lr*w6*in1_val*sum_
            w3 += lr*w5*in2_val*sum_
            w4 += lr*w6*in2_val*sum_
            output = pred1*(w1*w5 + w2*w6) + pred2*(w3*w5 + w4*w6)
            if j == 0 and k == 0:
                print(f"""After a single data point our RMSE is {np.sqrt(mse(y, output.flatten()))}""")
        print(f"""After {k+1} epochs our RMSE is {np.sqrt(mse(y, output.flatten()))}""")
    return w1, w2, w3, w4, w5, w6

In [16]:
stoch_grad_desc(times_thru=5)

After a single data point our RMSE is 132.0718494436695
After 1 epochs our RMSE is 130.71260165965495
After 2 epochs our RMSE is 125.90039567221291
After 3 epochs our RMSE is 114.90068010279187
After 4 epochs our RMSE is 105.45570922177163
After 5 epochs our RMSE is 102.94826034137003


(array([3.22727686]),
 array([8.73560832]),
 array([0.87579055]),
 array([0.97998175]),
 array([3.12714953]),
 array([8.44527613]))

## Easy Regression Problem

Our error has decreased with more epochs, but to illustrate the full power of our network let's see if it can find the right coefficients for an "easy" problem where there is a strong correlation between both of two input features and the target.

In [17]:
# Setting up the problem

X_easy, y_easy = make_regression(n_features=2, n_informative=2,
                                 random_state=42)

In [18]:
# Again defining weights randomly. We'll be using the same network,
# so we need six weights.

np.random.seed(42)

w_easy = np.random.rand(6)

In [19]:
# Applying our stoch_grad_desc() function

final_weights = stoch_grad_desc(pred1=X_easy[:, 0],
                                pred2=X_easy[:, 1],
                                y=y_easy,
         w1=w_easy[0], w2=w_easy[1], w3=w_easy[2],
         w4=w_easy[3], w5=w_easy[4], w6=w_easy[5],
         lr=4e-3, times_thru=3)
final_weights

After a single data point our RMSE is 105.2827433708648
After 1 epochs our RMSE is 0.0003058349135338329
After 2 epochs our RMSE is 5.040642635748519e-09
After 3 epochs our RMSE is 1.2706999512590757e-13


(6.556340807199361,
 10.75555705679945,
 6.914105827696129,
 7.997420739574159,
 4.334717820507707,
 5.515048597227902)

### Comparing with `LinearRegression()`

We can translate these final weight values into $\beta_1$ and $\beta_2$ for a traditional linear regression $\hat{y} = \beta_1X_1 + \beta_2X_2$.

Above we calculated $\hat{y} = X_1(w_1w_5 + w_2w_6) + X_2(w_3w_5 + w_4w_6)$.

Thus we have:

- $\beta_1 = w_1w_5 + w_2w_6$, and
- $\beta_2 = w_3w_5 + w_4w_6$.

Plugging these in for our final calculated weights we have:

In [20]:
beta1 = final_weights[0]*final_weights[4] +\
    final_weights[1]*final_weights[5]
beta2 = final_weights[2]*final_weights[4] +\
    final_weights[3]*final_weights[5]
print(f"Our mini-NN found coefficients of {np.round(beta1, 8)}\
 and {np.round(beta2, 8)}.")

Our mini-NN found coefficients of 87.73730719 and 74.07686178.


Let's compare these numbers with the results of `LinearRegression()`:

In [21]:
LinearRegression(fit_intercept=False).fit(X_easy, y_easy).coef_

array([87.73730719, 74.07686178])

Our little neural network, which has:
- only five neurons;
- randomly generated initial weights;
- only linear activation functions; and
- all biases set to 0

can do the same job as a linear regression after just a couple of epochs!